In [7]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import LeakyReLU

import keras_tuner
from kerastuner.tuners import RandomSearch
from tensorflow.keras.callbacks import EarlyStopping, Callback

!pip install shap
import shap
import pickle
import copy

import seaborn as sns; sns.set_theme()
from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib as mlp

import lime
import lime.lime_tabular

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.


In [2]:
inp = pd.read_csv("/kaggle/input/mamu-data/Mamu.csv", header = 0, low_memory=False, sep=",")
inp.index = range(len(inp))
inp.head()

inp = inp.groupby('Sequence', as_index=False).mean()
inp

,Sequence,A001,A002,A008,B008,B017
0,AAADTGHS,0.0787,0.425,0.420,0.379,0.1230
1,AAADTGHSN,0.0063,0.478,0.066,0.133,0.1070
2,AAADTGHSNQ,0.1410,0.737,0.253,0.284,0.1880
3,AAAPGLGR,0.2490,0.913,0.676,0.859,0.1700
4,AAAQHRGP,0.1520,0.349,0.198,0.420,0.0338
...,...,...,...,...,...,...
61061,YYTPKIVGG,0.9690,1.370,1.030,0.693,0.3130
61062,YYTPKIVGGI,1.9000,1.630,0.797,0.632,0.3940
61063,YYVDGSCN,0.0339,0.612,0.128,0.135,0.2580
61064,YYVDGSCNK,0.2720,1.150,0.320,0.800,0.3950


In [4]:
#Tạo list các axit amin để encoding
peptides = inp['Sequence'].tolist()
peptides.sort()
vocab = set(''.join([str(i) for i in peptides]))
vocab.add('END')
vocab_list = list(vocab)
vocab_list.sort()
#vocab_list

#Thêm END vào các peptides dài 8-9 có độ dài 10 
char_index = dict((vocab_list[i], i) for i in range(len(vocab_list)))
X = []
x_name = [str(i)[0:10] for i in peptides]
for i in x_name:
    tmp = [char_index[j] for j in str(i)]
    for k in range(0,10 - len(str(i))):
        tmp.append(char_index["END"])
    X.append(tmp)
#X

#Tạo list chứa toàn bộ giá trị liên kết Mamu
my_list=[]
Y = []
for index, row in inp.iterrows():
    my_list.append(row.A001)
    my_list.append(row.A002)
    my_list.append(row.A008)
    my_list.append(row.B008)
    my_list.append(row.B017)
    Y.append(my_list)
    my_list=[]
#all_int_list[0]

X = np.asarray(X)
Y = np.asarray(Y)

In [ ]:
#Chia thành tập train, test, val
x_train_all, x_test, y_train_all, y_test = train_test_split(X,Y, test_size=0.1, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, test_size=0.2, random_state=42)

In [ ]:
np.savetxt('CCS_x_test.txt', x_test)
np.savetxt('CCS_y_test.txt', y_test)
np.savetxt('CCS_x_train.txt', x_train)
np.savetxt('CCS_y_train.txt', y_train)
np.savetxt('CCS_x_val.txt', x_val)
np.savetxt('CCS_y_val.txt', y_val)

In [6]:
x_train = np.loadtxt('/kaggle/input/mamu-data/Mamu_x_train.txt')

x_test = np.loadtxt('/kaggle/input/mamu-data/Mamu_x_test.txt')

y_train = np.loadtxt('/kaggle/input/mamu-data/Mamu_y_train.txt')

y_test = np.loadtxt('/kaggle/input/mamu-data/Mamu_y_test.txt')

x_val = np.loadtxt('/kaggle/input/mamu-data/Mamu_x_val.txt')

y_val = np.loadtxt('/kaggle/input/mamu-data/Mamu_y_val.txt')

In [8]:
%%time

#Model LSTM 
model = Sequential()
model.add(Embedding(output_dim = 50, input_dim = 21, input_length = 10))

model.add(LSTM(128,return_sequences=True, input_shape=(10,21)))
model.add(Dropout(0.48386820999386737))
model.add(LSTM(128,return_sequences=False))
model.add(Dropout(0.1829057341070159))

model.add(Dense(64))
model.add(LeakyReLU(alpha=0.3))
model.add(Dropout(0.11769401187577534))

model.add(Dense(5))

optimizermodel = tf.keras.optimizers.Adam(0.001)
optimizermodel.learning_rate.assign(0.005)
model.compile(loss='mse', optimizer = optimizermodel, metrics=['mse'])

hist = model.fit(x_train, y_train,
                batch_size = 5000,
                epochs = 200,
                validation_data = (x_val, y_val))



Epoch 1/200


Argument `input_length` is deprecated. Just remove it.
Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
Argument `alpha` is deprecated. Use `negative_slope` instead.


9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 131ms/step - loss: 0.6681 - mse: 0.6681 - val_loss: 0.3915 - val_mse: 0.3915
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.3942 - mse: 0.3942 - val_loss: 0.3386 - val_mse: 0.3386
Epoch 3/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.3345 - mse: 0.3345 - val_loss: 0.3000 - val_mse: 0.3000
Epoch 4/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.3036 - mse: 0.3036 - val_loss: 0.2794 - val_mse: 0.2794
Epoch 5/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.2803 - mse: 0.2803 - val_loss: 0.2589 - val_mse: 0.2589
Epoch 6/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.2635 - mse: 0.2635 - val_loss: 0.2443 - val_mse: 0.2443
Epoch 7/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 0.2481 - mse: 0.2481 - val_loss: 0.2292 - val_mse: 0.2292
Epoch 8/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.2312 - mse: 0.2312 - val_loss: 0.2166 - val_mse: 0.2166
Epoch 9/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - loss: 0.2222 - mse: 0.2222 - va

In [9]:
y_pred = model.predict(x_test)
model_r2 = r2_score(y_test, y_pred)
model_mae = mean_absolute_error(y_test, y_pred)
model_mse = mean_squared_error(y_test, y_pred)
print(model_r2, model_mae, model_mse)

191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
0.892146989655747 0.13075374257907096 0.03867312538641852


In [11]:
def build_model(hp):
    model = Sequential()
    model.add(Embedding(output_dim=50, input_dim=21, input_length=10))

    model.add(GRU(hp.Int('units_1', min_value=32, max_value=128, step=32),
                  return_sequences=True, input_shape=(10, 21)))
    model.add(Dropout(hp.Float('dropout_1', min_value=0.0, max_value=0.8)))

    model.add(GRU(hp.Int('units_2', min_value=32, max_value=128, step=32)))
    model.add(Dropout(hp.Float('dropout_2', min_value=0.0, max_value=0.8)))

    model.add(Dense(1))

    learning_rate = hp.Choice('learning_rate', values=[0.001, 0.005, 0.01, 0.05])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])

    return model

tuner = RandomSearch(
    build_model,
    objective='val_mse',
    max_trials=20,  
    executions_per_trial=1,  
    directory='tuner_results',
    project_name='gru_model_tuning_2'
)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

tuner.search(x_train, y_train,
             epochs=200,
             batch_size=tuner.oracle.hyperparameters.Choice('batch_size', values=[64, 128, 256, 512, 1024, 2048, 4096]),
             validation_data=(x_val, y_val),
             callbacks=[early_stopping],
             verbose=1)

best_model = tuner.get_best_models(num_models=1)[0]
best_run = tuner.oracle.get_best_trials(num_trials=1)[0]

best_model.summary()
print(best_run.hyperparameters.values)


Trial 20 Complete [00h 01m 17s]
val_mse: 0.3385644555091858

Best val_mse So Far: 0.33092495799064636
Total elapsed time: 00h 42m 06s


Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 20 variables. 


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 10, 50)         │         1,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 10, 128)        │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 128)            │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 169,371 (661.61 KB)

 Trainable params: 169,371 (661.61 KB)

 Non-trainable params: 0 (0.00 B)

{'units_1': 128, 'dropout_1': 0.6407290410000607, 'units_2': 128, 'dropout_2': 0.1683099603544756, 'learning_rate': 0.001, 'batch_size': 256}


In [20]:
%%time

#model GRU
model1 = Sequential()
model1.add(Embedding(output_dim = 50, input_dim = 21, input_length = 10))

model1.add(GRU(128, return_sequences=True))
model1.add(Dropout(0.6407290410000607))

model1.add(GRU(128))
model1.add(Dropout(0.1683099603544756))

model1.add(Dense(5))

optimizer = tf.keras.optimizers.Adam(0.001)

model1.compile(loss='mse', optimizer=optimizer)
model1.fit(x_train, y_train, 
           epochs = 200, 
           batch_size = 512, 
           validation_data = (x_val, y_val))

Epoch 1/200
86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.4292 - val_loss: 0.3015
Epoch 2/200
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2885 - val_loss: 0.2547
Epoch 3/200
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2502 - val_loss: 0.2211
Epoch 4/200
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.2208 - val_loss: 0.1883
Epoch 5/200
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1918 - val_loss: 0.1685
Epoch 6/200
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1696 - val_loss: 0.1462
Epoch 7/200
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1506 - val_loss: 0.1275
Epoch 8/200
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1341 - val_loss: 0.1194
Epoch 9/200
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1257 - val_loss: 0.1072
Epoch 10/200
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1158 - val_loss: 0.1014
Epoch 11/200
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1097 - val_loss: 0.0981
Epoch 12/200
86/86 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.

In [21]:
y_pred1 = model1.predict(x_test)
model1_r2 = r2_score(y_test, y_pred1)
model1_mae = mean_absolute_error(y_test, y_pred1)
model1_mse = mean_squared_error(y_test, y_pred1)
print(model1_r2, model1_mae, model1_mse)

191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
0.8891964334183582 0.13199357773490053 0.039643745431998216


In [ ]:
#Chọn model
model = model1

**SHAP**

In [ ]:
%%time

#Tạo 50 cụm dữ liệu để tính toán SHAP
x_trainmeans = shap.kmeans(np.asarray(x_train), 50)
explainer = shap.KernelExplainer(model.predict, x_trainmeans)
shap_values = explainer.shap_values(np.asarray(x_test))

#lưu giá trị SHAP
with open("Mamu_shapvalues_GRU.pkl","wb") as f:
    pickle.dump(shap_values, f)

In [ ]:
with open("/kaggle/input/mamu-data/shapvaluesMamu.pkl","rb") as f:
    shap_values = pickle.load(f)

**LIME**

In [ ]:
%%time
def predict_function(model, data, output_index):
    prediction = model.predict(data)
    return prediction[:,output_index]
    
#Định nghĩa hàm giải thích LIME
explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data = x_train,
    mode = 'regression',
    feature_names = [f'Pos{i+1}' for i in range(10)],
)
positions = [f'Pos{i+1}' for i in range(10)]

lime_values = []
for i in range(len(x_test)):
    x_instance = x_test[i]
    instance_contributions = []
    #Tính LIME cho từng instance với từng output
    for j in range(5):
        predict_fn = lambda x: predict_function(model, x, output_index=j)
        exp = explainer.explain_instance(
            data_row = x_instance,
            predict_fn = predict_fn
        )

        ordered_contributions = {pos: 0.0 for pos in positions}

    for feature_value, contribution in exp.as_list():
        pos = next((word for word in feature_value.split() if word.startswith('Pos')), None)
        if pos in ordered_contributions:
            ordered_contributions[pos] += contribution

    contributions_sorted = [ordered_contributions[pos] for pos in positions]
    instance_contributions.append(contributions_sorted)
lime_values.append(instance_contributions)

with open('Mamu_lime_values_LSTM.pkl', 'wb') as f:
    pickle.dump(lime_values, f)

In [ ]:
with open('/kaggle/input/mamu-data/Mamu_lime_values_LSTM.pkl', 'rb') as f:
    lime_values = pickle.load(f)

**OSA**

In [24]:
%%time

#Tính OSA
baseline = 0
original_output = model.predict(x_test)
osa_values = np.zeros((5, len(x_test), 10), dtype=np.float32)

for i in range(10):
    osa_inputs = x_test.copy()
    osa_inputs[:,i] = baseline
    osa_output = model.predict(osa_inputs)

    for j in range(5):
        osa_values[j,:,i] = np.abs(original_output[:,j] - osa_output[:,j])



191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
CPU times: user 6.76 s, sys: 587 ms, total: 7.35 s
Wall time: 4.99 s


In [33]:
print(len(osa_values))

5


**HEATMAP**

In [ ]:
#Tạo mange lưu các giá trị SHAP/LIME/OSA của 1 axit amin ở cùng 1 vị trí (pos=[1,10])

#Chọn giá trị
explain_values = osa_values
x = 0
while x < 5:
    if x==0:
        figfilename='MeanLIMEValA001-'
        allelename = 'A001'
    if x==1:
        figfilename='MeanLIMEValA002-'
        allelename = 'A002'
    if x==2:
        figfilename='MeanLIMEValA008-'
        allelename = 'A008'
    if x==3:
        figfilename='MeanLIMEValB008-'
        allelename = 'B008'
    if x==4:
        figfilename='MeanLIMEValB017-'
        allelename = 'B017'

    aa = []
    i = 0
    while i < 10:
        aa.append([[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]])
        i+=1

    i=0
    while i < len(x_test):
        j = 0
        while j < len(x_test[i]):
            aa[j][int(x_test[i][j])].append(explain_values[x][i][j])
            j+=1
        i+=i

    heatmap = []

    i = 0
    while i < len(aa):
        j = 0
        heatmap.append([])
        while j < len(aa[i]):
            if len(aa[i][j]) > 1:
                aa[i][j] = aa[i][j][1:]
                heatmap[i].append(sum(aa[i][j])/float(len(aa[i][j])))
            else:
                heatmap[i].append(0)
            j+=1
        i+=1
    heatmap = np.array(heatmap)


    plt.figure()
    figfilename = figfilename
    x_axis_labels = ['A','C','D','E','End','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y'] # labels for x-axis
    y_axis_labels = ['1','2','3','4','5','6','7','8','9','10']
    sns.set(font_scale = 2)

    rcParams['figure.figsize'] = 20,16
    ax = sns.heatmap(heatmap,xticklabels=x_axis_labels, yticklabels=y_axis_labels,linewidths=.5,  cmap="viridis")

    ax.set(xlabel='Amino Acid', ylabel='Position', title='Mean SHAP Values' + allelename)
    ax.figure.savefig(figfilename +'LSTM.png')
    plt.show()

    x+=1